# CS295 Final Project 
Stanhope Nwosu 

Calculating the variance without revealing the mean via BGW Protocol. 

In [10]:
# Imports and definitions
import numpy as np
from collections import defaultdict
from collections import namedtuple
import urllib.request

_PRIME = 2 ** 13 - 1

shamir_lib_url = "https://raw.githubusercontent.com/jnear/cs295-secure-computation/master/utils/shamir.py"
### DANGER: this line is dangerous. Make sure the URL above is correct, and has correct code.
exec(urllib.request.urlopen(shamir_lib_url).read())

def share_shamir(t, n, x, prime=_PRIME):
    shares_with_x = share_input(x, minimum=t, shares=n, prime=prime)
    return [y for x,y in shares_with_x]

def reconstruct_shamir(shares, prime=_PRIME):
    shares_with_x = list(zip(range(1, len(shares)+1), shares))
    return recover_secret(shares_with_x, prime=prime)

class Party:
    """A participant in a multiparty computation protocol."""
    def __init__(self, field_size=_PRIME):
        """Initialize the field size and dictionary to hold received messages."""
        self.field_size = field_size
        self.input = None
        self.output = None
        self.received = defaultdict(list)

    def send(self, other, round, msg):
        """Simulate sending a message `msg` to another party `other` during round `round`"""
        other.received[round].append(msg)
    def get_view(self):
        """Returns the view of this party: its input, output, and received messages."""
        return (self.input, self.output, dict(self.received))

In [11]:
AddGate = namedtuple('AddGate', ['in1', 'in2'])
MultGate = namedtuple('MultGate', ['in1', 'in2'])
largest_wire = 0

def new_wire():
    global largest_wire
    largest_wire += 1
    return f'w{largest_wire}'

In [3]:
class MeanParty(Party):
    def round1(self, parties, input_num):
        self.input = input_num
        self.parties = parties
        n = len(parties)
        t = n-1
        ### BEGIN SOLUTION
        shares = share_shamir(t, n, input_num)
        for p, s in zip(self.parties, shares):
            self.send(p, 1, s)
        ### END SOLUTION
        
    def round2(self):
        ### BEGIN SOLUTION
        s = sumFE(_PRIME, self.received[1])
        for p in self.parties:
            self.send(p, 2, s)
        ### END SOLUTION

    def round3(self):
        ### BEGIN SOLUTION
        shares = self.received[2]
        total_sum = reconstruct_shamir(shares)
        mean = (1/len(self.parties)) * total_sum
        self.output = mean
        ### END SOLUTION

In [7]:
# TEST CASE for question 1(b)
NUM_PARTIES = 10
parties = [MeanParty() for _ in range(NUM_PARTIES)]
nums = [np.random.randint(0, 10) for _ in range(NUM_PARTIES)]
for p,n in zip(parties, nums):
    p.round1(parties, n)
for p in parties:
    p.round2()
for p in parties:
    p.round3()
# for p in parties:
#     print(p.get_view())
    
assert np.abs(p.output - np.mean(nums)) < 1

In [5]:
#Variance Method
class VarianceParty(Party):
    def round1(self, parties, input_num):
        self.input = input_num
        self.parties = parties
        n = len(parties)
        t = n-1
        ### BEGIN SOLUTION
        shares = share_shamir(t, n, input_num)
        for p, s in zip(self.parties, shares):
            self.send(p, 1, s)
        ### END SOLUTION
    def round2(self):
        ### BEGIN SOLUTION
        s = sumFE(_PRIME, self.received[1])
        for p in self.parties:
            self.send(p, 2, s)
        ### END SOLUTION

    def round3(self):
        ### BEGIN SOLUTION
        shares = self.received[2]
        total_sum = reconstruct_shamir(shares)
        mean = (1/len(self.parties)) * total_sum
        # now calc the squared distance from the mean
        # and share it out
        my_sq_dist = int((self.input - mean)**2) % _PRIME
        n = len(parties)
        t = n-1
        shares = share_shamir(t, n, my_sq_dist)
        for p, s in zip(self.parties, shares):
            self.send(p, 3, s)
        ### END SOLUTION

    def round4(self):
        ### BEGIN SOLUTION
        s = sumFE(_PRIME, self.received[3])
        for p in self.parties:
            self.send(p, 4, s)
        ### END SOLUTION
    def round5(self):
        ### BEGIN SOLUTION
        shares = self.received[4]
        total_sq_sum = reconstruct_shamir(shares)
        total_variance = total_sq_sum / len(self.parties)
        self.output = total_variance

In [6]:
# TEST CASE for question 2(b)
NUM_PARTIES = 10
parties = [VarianceParty() for _ in range(NUM_PARTIES)]
nums = [np.random.randint(0, 50) for _ in range(NUM_PARTIES)]
for p,n in zip(parties, nums):
    p.round1(parties, n)
for p in parties:
    p.round2()
for p in parties:
    p.round3()
for p in parties:
    p.round4()
for p in parties:
    p.round5()
for p in parties:
    #print(p.get_view())
    assert np.abs(p.output - np.var(nums)) <= 1

In [15]:
#BGW Protocol
class BGWParty(Party):
    def receive_inputs(self, input_wire_values, circuit, eval_order, t, n):
        self.wire_values = input_wire_values
        self.circuit = circuit
        self.eval_order = eval_order
        self.is_done = False
        self.t = t
        self.n = n

    def round_n(self, round_num, parties):
        """Perform one round of the BGW protocol. Reference Section 3.3 in 'Pragmatic MPC.'"""
        ### BEGIN SOLUTION
        # if our last gate was multiplication, finish the degree reduction
        # degree reduction is always required except for the first round
        if round_num > 1:
            #print('doing degree reduction')
            h_i_js = self.received[round_num - 1]

            V = np.vander(range(1, self.n+1), increasing=True)
            Vi = inversematrix(V, _PRIME)
            lambda_js = list(np.asarray(Vi)[0])

            prods = [multFE(_PRIME, lambda_j, s) for lambda_j, s in zip(lambda_js)]
            s_i = sumFE(_PRIME, prods)

            self.wire_values[self.last_wire] = s_i
        while True:
            # if there is nothing left to evaluate, then we are done
            if self.eval_order == []:
                self.is_done = True
                return

            # find the next wire to evaluate
            next_wire = self.eval_order.pop(0)
            #print('evaluating wire', next_wire)
                                                                        
            # find the gate for that wire
            next_gate = self.circuit.pop(next_wire)
                                                                        
            if isinstance(next_gate, AddGate):
                w1 = self.wire_values[next_gate.in1]
                w2 = self.wire_values[next_gate.in2]
                w_out = plusFE(_PRIME, w1, w2)
                self.wire_values[next_wire] = w_out
                                                                        
            elif isinstance(next_gate, MultGate):
                self.last_wire = next_wire
                w1 = self.wire_values[next_gate.in1]
                w2 = self.wire_values[next_gate.in2]
                s_i = multFE(_PRIME, w1, w2)
                h_is = share_shamir(self.t, self.n, s_i)
                                                                        
                for p, h_i_j in zip(parties, h_is):
                    self.send(p, round_num, h_i_j)
                return
        ### END SOLUTION

In [22]:
#Running BGW Protocol
def run_bgw(inputs, circuit, eval_order, output_wires):
    n = 6
    t = 3

    # calculate the input shares
    input_shares = {w: share_shamir(t, n, x) for w,x in inputs.items()}
    parties = [BGWParty(_PRIME) for _ in range(n)]

    # split the shares up for the parties
    keys = input_shares.keys()
    party_shares = [dict(zip(keys, vals)) for vals in zip(*(input_shares[keys]))]
    
    # kick off each party with its inputs and copies of the circuit and evaluation plan
    for p, s in zip(parties, party_shares):
        p.receive_inputs(s, circuit.copy(), eval_order.copy(), t, n)

    done = False
    round_num = 1

    # keep evaluating until one of the parties is finished
    while not done:
        for p in parties:
            p.round_n(round_num, parties)
            if p.is_done:
                done = True
        round_num = round_num + 1
    
    # for each output wire, get the shares from the parties for that wire
    output_shares = [[p.wire_values[w] for p in parties] for w in output_wires]
    outputs = [reconstruct_shamir(shares) for shares in output_shares]
    return outputs

In [23]:
# TEST CASE: a simple circuit
inputs = {'x': 5, 'y': 6}
circuit = {'w1': AddGate('x', 'y'),
 'w2': MultGate('w1', 'x'),
 'w3': AddGate('w2', 'w1')}
eval_order = list(circuit.keys())
result = run_bgw(inputs, circuit, eval_order, ['w3'])
print('Result:', result)
assert result == [66]

TypeError: unhashable type: 'dict_keys'